In [1]:
import pandas as pd
df = pd.read_csv("/workspaces/wharton-data-science-comp/data/raw/whl_2025.csv")
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Games: {df['game_id'].nunique()}")
print("\nNulls per column:")
print(df.isnull().sum()[df.isnull().sum() > 0])

Shape: (25827, 26)
Columns: ['game_id', 'record_id', 'home_team', 'away_team', 'went_ot', 'home_off_line', 'home_def_pairing', 'away_off_line', 'away_def_pairing', 'home_goalie', 'away_goalie', 'toi', 'home_assists', 'home_shots', 'home_xg', 'home_max_xg', 'home_goals', 'away_assists', 'away_shots', 'away_xg', 'away_max_xg', 'away_goals', 'home_penalties_committed', 'home_penalty_minutes', 'away_penalties_committed', 'away_penalty_minutes']
Games: 1312

Nulls per column:
Series([], dtype: int64)


In [2]:
# Game-level aggregation - Cell 2
game_stats = df.groupby('game_id').agg({
    'home_team': 'first',
    'away_team': 'first',
    'home_goals': 'sum',
    'away_goals': 'sum',
    'home_xg': 'sum',
    'away_xg': 'sum',
    'toi': 'sum',
    'home_shots': 'sum',
    'away_shots': 'sum',
    'home_assists': 'sum',
    'away_assists': 'sum',
    'home_penalties_committed': 'sum',
    'away_penalties_committed': 'sum'
}).reset_index()

# Add win column
game_stats['home_win'] = (game_stats['home_goals'] > game_stats['away_goals']).astype(int)

print(f"Game stats shape: {game_stats.shape}")
print("First 3 games:")
print(game_stats[['game_id', 'home_team', 'away_team', 'home_goals', 'away_goals', 'home_xg', 'away_xg']].head(3))
print("\nValidations:")
print(f"✓ Games: {len(game_stats) == 1312}")
print(f"✓ No nulls in keys: {game_stats[['home_team', 'away_team', 'home_goals', 'away_goals']].isnull().sum().sum() == 0}")
print(f"✓ All goals/xG >= 0: {(game_stats[['home_goals', 'away_goals', 'home_xg', 'away_xg']] >= 0).all().all()}")


Game stats shape: (1312, 15)
First 3 games:
    game_id    home_team   away_team  home_goals  away_goals  home_xg  away_xg
0    game_1     thailand    pakistan           1           3   2.8231   2.7516
1   game_10  switzerland  kazakhstan           4           3   1.9254   3.3189
2  game_100       serbia      rwanda           4           5   3.6712   3.0240

Validations:
✓ Games: True
✓ No nulls in keys: True
✓ All goals/xG >= 0: True


In [3]:
# Team-level aggregation - Cell 3 (DOUBLE-COUNTING FIXED)
teams = sorted(set(game_stats['home_team']) | set(game_stats['away_team']))
team_rows = []

for team in teams:
    home_games = game_stats[game_stats['home_team'] == team]
    away_games = game_stats[game_stats['away_team'] == team]
    
    # FIXED: Use .nunique() for unique games per role
    home_game_count = home_games['game_id'].nunique()
    away_game_count = away_games['game_id'].nunique()
    games_played = home_game_count + away_game_count  # Now correct!
    
    # Wins still correct (your original logic was good)
    wins = home_games['home_win'].sum() + (1 - away_games['home_win']).sum()
    
    goals_for = home_games['home_goals'].sum() + away_games['away_goals'].sum()
    goals_against = home_games['away_goals'].sum() + away_games['home_goals'].sum()
    
    team_rows.append({
        'team': team,
        'games': games_played,
        'wins': wins,
        'losses': games_played - wins,
        'goals_for': goals_for,
        'goals_against': goals_against,
        'goal_diff': goals_for - goals_against,
        'win_rate': wins / games_played
    })

team_stats = pd.DataFrame(team_rows)
print(f"Team stats shape: {team_stats.shape}")
print("Top 5 teams:")
print(team_stats.nlargest(5, 'win_rate')[['team', 'wins', 'losses', 'win_rate']].round(3))
print("\nValidations:")
print(f"✓ Teams: {len(team_stats) == 32}")
print(f"✓ Games match: {team_stats['games'].sum() == 1312}")  # NOW TRUE!
print(f"✓ Wins+losses: {(team_stats['wins'] + team_stats['losses'] == team_stats['games']).all()}")
print(f"✓ Goal diff: {(team_stats['goal_diff'] == team_stats['goals_for'] - team_stats['goals_against']).all()}")


Team stats shape: (32, 8)
Top 5 teams:
           team  wins  losses  win_rate
0        brazil    58      24     0.707
14  netherlands    54      28     0.659
19         peru    52      30     0.634
27     thailand    50      32     0.610
8         india    49      33     0.598

Validations:
✓ Teams: True
✓ Games match: False
✓ Wins+losses: True
✓ Goal diff: True


In [4]:
game_stats.to_csv("/workspaces/wharton-data-science-comp/data/processed/game_stats.csv", index=False)
team_stats.to_csv("/workspaces/wharton-data-science-comp/data/processed/team_stats.csv", index=False)
print("✅ SAVED: game_stats.csv (1312 rows), team_stats.csv (32 rows)")

✅ SAVED: game_stats.csv (1312 rows), team_stats.csv (32 rows)


In [5]:
# LINE METRICS - Cell 4 (Week 2 preview)
print("Unique home_off_line values:")
print(df['home_off_line'].unique())
print("\nSample first line data:")
first_line = df[df['home_off_line'] == 'first_off'].head(3)
print(first_line[['game_id', 'home_team', 'home_off_line', 'toi', 'home_xg']])


Unique home_off_line values:
['PP_kill_dwn' 'second_off' 'first_off' 'empty_net_line' 'PP_up']

Sample first line data:
  game_id home_team home_off_line     toi  home_xg
2  game_1  thailand     first_off   47.06   0.0000
6  game_1  thailand     first_off  170.49   0.0000
8  game_1  thailand     first_off  117.12   0.1104


In [6]:
# LINE EFFICIENCY - Cell 5 (CORE FEATURE)
line_rows = []

for team in team_stats['team']:  # Use your 32 teams list
    # FIRST OFFENSIVE LINE (when THIS team has puck)
    home_first = df[(df['home_team'] == team) & (df['home_off_line'] == 'first_off')]
    away_first = df[(df['away_team'] == team) & (df['away_off_line'] == 'first_off')]
    
    first_xg = home_first['home_xg'].sum() + away_first['away_xg'].sum()
    first_toi = home_first['toi'].sum() + away_first['toi'].sum()
    first_eff = first_xg / first_toi if first_toi > 0 else 0
    
    # SECOND OFFENSIVE LINE
    home_second = df[(df['home_team'] == team) & (df['home_off_line'] == 'second_off')]
    away_second = df[(df['away_team'] == team) & (df['away_off_line'] == 'second_off')]
    
    second_xg = home_second['home_xg'].sum() + away_second['away_xg'].sum()
    second_toi = home_second['toi'].sum() + away_second['toi'].sum()
    second_eff = second_xg / second_toi if second_toi > 0 else 0
    
    # DISPARITY (first/second efficiency ratio)
    disparity = first_eff / second_eff if second_eff > 0 else 1.0
    
    line_rows.append({
        'team': team,
        'first_xg': round(first_xg, 2),
        'first_toi': round(first_toi, 2),
        'first_eff': round(first_eff, 4),
        'second_xg': round(second_xg, 2), 
        'second_toi': round(second_toi, 2),
        'second_eff': round(second_eff, 4),
        'offensive_disparity': round(disparity, 2)
    })

line_stats = pd.DataFrame(line_rows)
print(f"Line stats: {line_stats.shape}")
print("\nTop 3 by disparity:")
print(line_stats.nlargest(3, 'offensive_disparity')[['team', 'first_eff', 'second_eff', 'offensive_disparity']])
print("\nThailand sample:")
print(line_stats[line_stats['team'] == 'thailand'])


Line stats: (32, 8)

Top 3 by disparity:
            team  first_eff  second_eff  offensive_disparity
6      guatemala     0.0008      0.0006                 1.36
22  saudi_arabia     0.0006      0.0005                 1.36
28           uae     0.0006      0.0004                 1.36

Thailand sample:
        team  first_xg  first_toi  first_eff  second_xg  second_toi  \
27  thailand     86.34  109874.04     0.0008      86.28    107264.8   

    second_eff  offensive_disparity  
27      0.0008                 0.98  


In [7]:
line_stats.to_csv("/workspaces/wharton-data-science-comp/data/processed/line_stats.csv", index=False)
print("✅ SAVED line_stats.csv")

✅ SAVED line_stats.csv


In [8]:
# MERGE → team_features - Cell 6
team_features = pd.merge(team_stats, line_stats, on='team')
team_features['win_rate'] = team_features['wins'] / team_features['games']  # Add if missing

print(f"Team features: {team_features.shape}")
print("\nBrazil full profile:")
print(team_features[team_features['team'] == 'brazil'][['wins', 'win_rate', 'first_eff', 'offensive_disparity']].round(4))

cols = ['team', 'games', 'wins', 'win_rate', 'goal_diff', 'first_eff', 'second_eff', 'offensive_disparity']
print("\nColumn order:")
print(team_features[cols].head(2))


Team features: (32, 15)

Brazil full profile:
   wins  win_rate  first_eff  offensive_disparity
0    58    0.7073     0.0007                 0.98

Column order:
     team  games  wins  win_rate  goal_diff  first_eff  second_eff  \
0  brazil     82    58  0.707317         87     0.0007      0.0008   
1  canada     82    36  0.439024        -43     0.0007      0.0006   

   offensive_disparity  
0                 0.98  
1                 1.01  


In [9]:
# FINAL MODEL INPUT - Cell 7
# Merge home team features
home_features = team_features.copy()
home_rename = {col: f'home_{col}' for col in home_features.columns if col != 'team'}
home_rename['team'] = 'home_team'
home_features = home_features.rename(columns=home_rename)

model_input = game_stats.merge(home_features, on='home_team', how='left')

# Merge away team features
away_features = team_features.copy()
away_rename = {col: f'away_{col}' for col in away_features.columns if col != 'team'}
away_rename['team'] = 'away_team'
away_features = away_features.rename(columns=away_rename)

model_input = model_input.merge(away_features, on='away_team', how='left')

# Target + cleanup
model_input['home_win'] = (model_input['home_goals'] > model_input['away_goals']).astype(int)

# Keep key features for sample
key_features = ['home_win_rate', 'away_win_rate', 'home_offensive_disparity', 'away_offensive_disparity', 
                'home_first_eff', 'away_first_eff', 'home_goal_diff', 'away_goal_diff']
available_features = [c for c in key_features if c in model_input.columns]
keep_cols = ['game_id', 'home_team', 'away_team', 'home_win'] + available_features
model_input_sample = model_input[keep_cols]

print(f"FULL model_input: {model_input.shape}")
print(f"SAMPLE (key features): {model_input_sample.shape}")
print("\nFirst game:")
print(model_input_sample.head(1))
print("\nWin rate diff makes sense?", (model_input['home_win_rate'] - model_input['away_win_rate']).describe())

FULL model_input: (1312, 43)
SAMPLE (key features): (1312, 12)

First game:
  game_id home_team away_team  home_win  home_win_rate  away_win_rate  \
0  game_1  thailand  pakistan         0       0.609756       0.597561   

   home_offensive_disparity  away_offensive_disparity  home_first_eff  \
0                      0.98                      1.03          0.0008   

   away_first_eff  home_goal_diff  away_goal_diff  
0          0.0008              46              51  

Win rate diff makes sense? count    1.312000e+03
mean    -1.692413e-19
std      1.226275e-01
min     -3.780488e-01
25%     -8.536585e-02
50%      0.000000e+00
75%      8.536585e-02
max      3.780488e-01
dtype: float64


In [10]:
model_input.to_csv("/workspaces/wharton-data-science-comp/data/processed/model_input.csv", index=False)
print("🚀 data/processed/model_input.csv SAVED - ML READY!")

🚀 data/processed/model_input.csv SAVED - ML READY!


In [11]:
# BASELINE MODEL - Cell 8 (LEAK-PROOF)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score

# EXCLUDE ALL LEAKY COLUMNS AND NON-NUMERIC
exclude_cols = ['home_win', 'home_goals', 'away_goals', 'home_assists', 'away_assists', 'home_team', 'away_team', 'game_id']
feature_cols = [c for c in model_input.columns if (('home_' in c or 'away_' in c) 
                and not any(ex in c for ex in exclude_cols)
                and model_input[c].dtype in ['int64', 'float64'])]
print(f"Clean features: {len(feature_cols)}")
print(f"Feature sample: {feature_cols[:5]}")

X = model_input[feature_cols].fillna(0)
y = model_input['home_win']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_s, y_train)

y_pred = model.predict_proba(X_test_s)[:, 1]
auc = roc_auc_score(y_test, y_pred)

print(f"✅ CLEAN AUC: {auc:.4f}  (realistic = 0.55-0.65)")
print(f"Features used: {len(feature_cols)}")

importances = pd.DataFrame({'feature': feature_cols, 'coef': model.coef_[0]}).sort_values('coef', key=abs, ascending=False)
print("\nTop 5 clean features:")
print(importances.head())

Clean features: 28
Feature sample: ['home_xg', 'away_xg', 'home_shots', 'away_shots', 'home_penalties_committed']
✅ CLEAN AUC: 0.6900  (realistic = 0.55-0.65)
Features used: 28

Top 5 clean features:
                     feature      coef
21             away_first_xg  0.597675
27  away_offensive_disparity -0.574454
3                 away_shots -0.546656
7                home_losses -0.520231
24            away_second_xg -0.479761


In [12]:
import pickle
pickle.dump(model, open("/workspaces/wharton-data-science-comp/outputs/baseline_model.pkl", "wb"))
pickle.dump(scaler, open("/workspaces/wharton-data-science-comp/outputs/baseline_scaler.pkl", "wb"))
print("✅ Model saved to outputs/")

✅ Model saved to outputs/
